# Access to Hook services 

**Licence**: MIT <br>

This Notebook is going to cover topics like:  
* Authentication do Hook services  
* Get a list of available Hooks (Processors)  
* Trigger a data-harvesting hook - to donwload data to temporary s3 bucket

In [1]:
import json
from io import BytesIO
from urllib.parse import urlencode
import getpass
import pycurl
import requests
from IPython.display import JSON

## Autehentication - function

In [2]:
import requests
from lxml import html
from urllib.parse import parse_qs, urlparse

IAM_URL = "https://auth.destine.eu/"
CLIENT_ID = "dedl-hook"
REALM = "desp"
SERVICE_URL = "https://odp.data.destination-earth.eu/odata/v1/"


class DESPAuth:
    def __init__(self, username, password):
        self.username = username
        self.password = password

    def get_token(self):
        with requests.Session() as s:

            # Get the auth url
            auth_url = html.fromstring(s.get(url=IAM_URL + "/realms/" + REALM + "/protocol/openid-connect/auth",
                                     params = {
                                            "client_id": CLIENT_ID,
                                            "redirect_uri": SERVICE_URL,
                                            "scope": "openid",
                                            "response_type": "code"
                                     }
                                       ).content.decode()).forms[0].action
            
            # Login and get auth code
            login = s.post(auth_url,
                            data = {
                                "username" : self.username,
                                "password" : self.password,
                            },
                            allow_redirects=False
            )


            # We expect a 302, a 200 means we got sent back to the login page and there's probably an error message
            if login.status_code == 200:
                tree = html.fromstring(login.content)
                error_message_element = tree.xpath('//span[@id="input-error"]/text()')
                error_message = error_message_element[0].strip() if error_message_element else 'Error message not found'
                raise Exception(error_message)

            if login.status_code != 302:
                raise Exception("Login failed")
            

            auth_code = parse_qs(urlparse(login.headers["Location"]).query)['code'][0]

            # Use the auth code to get the token
            response = requests.post(IAM_URL + "/realms/" + REALM + "/protocol/openid-connect/token",
                    data = {
                        "client_id" : CLIENT_ID,
                        "redirect_uri" : SERVICE_URL,
                        "code" : auth_code,
                        "grant_type" : "authorization_code",
                        "scope" : ""
                    }
                )
            
            if response.status_code != 200:
                raise Exception("Failed to get token")

            token = response.json()['access_token']
        

            return token

class DEDLAuth:
    def __init__(self, desp_access_token):
        self.desp_access_token = desp_access_token

    def get_token(self):
        DEDL_TOKEN_URL='https://identity.data.destination-earth.eu/auth/realms/dedl/protocol/openid-connect/token'
        DEDL_CLIENT_ID='hda-public'
        AUDIENCE='hda-public'
        
        data = { 
            "grant_type": "urn:ietf:params:oauth:grant-type:token-exchange", 
            "subject_token": self.desp_access_token,
            "subject_issuer": "desp-oidc",
            "subject_token_type": "urn:ietf:params:oauth:token-type:access_token",
            "client_id": DEDL_CLIENT_ID,
            "audience": AUDIENCE
        }

        response = requests.post(DEDL_TOKEN_URL, data=data)
        
        print("Response code:", response.status_code)

        if response.status_code == 200: 
            dedl_token = response.json()["access_token"]
            return dedl_token
        else: 
            print(response.json())
            print("Error obtaining DEDL access token")
            
class AuthHandler:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.desp_access_token = None
        self.dedl_access_token = None
    
    def get_token(self):
        # Get DESP auth token
        desp_auth = DESPAuth(self.username, self.password)
        self.desp_access_token = desp_auth.get_token()
        
        # Get DEDL auth token
        dedl_auth = DEDLAuth(self.desp_access_token)
        self.dedl_access_token = dedl_auth.get_token()
        
        return self.dedl_access_token

## Authetication

In [3]:
DESP_USERNAME = ''
DESP_PASSWORD = ''

token = AuthHandler(DESP_USERNAME, DESP_PASSWORD)          
access_token = token.get_token()
 
# Check the status of the request
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token") 

Response code: 200
DEDL/DESP Access Token Obtained Successfully


## Get a list of avilable Hooks

In [7]:

api_headers = {'Authorization': 'Bearer ' + access_token}
service_root_url = "https://odp.data.destination-earth.eu/odata/v1/"
result = requests.get(service_root_url + "Workflows", headers=api_headers).json()

# Assuming 'result' is a JSON array
for item in result['value']:
    for key, value in item.items():
        print(f"{key}: {value}")
    print()  # Print an empty line to separate each item

Id: 2
Uuid: None
Name: lai
DisplayName: Sentinel-2: SNAP-Biophysical
Documentation: 
Description: The SNAP-BIOPHYSICAL processor derives vegetation biophysical variables based on top-of-canopy spectral reflectances from Sentinel-2 data. Following the approach described by Weiss et al. (2000, DOI: 10.1051/agro:2000105), the processor estimates: LAI (Leaf Area Index), FAPAR (Fraction of Absorbed Photosynthetically Active Radiation) and FVC (Fractional Vegetation Cover), all recognized as Essential Climate Variables (ECVs) by international organizations such as the Global Climate Observing System (GCOS)
InputProductType: S2MSI2A
InputProductTypes: ['S2MSI2A']
OutputProductType: None
OutputProductTypes: []
WorkflowVersion: 1.1.1
CustomInputSource: False
InputProductStatusOffline: False

Id: 4
Uuid: None
Name: card_bs_private
DisplayName: Sentinel-1: Terrain-corrected backscatter (Private)
Documentation: 
Description: Sentinel-1 CARD BS (Copernicus Analysis Ready Data Backscatter) processor

## Trigger a Hook - data harvest  
* Provide your username and password  
* Please be sure that 'source_realm' is set to 'desp'

In [9]:
test_run_id= "test_hook"
output_storage_url = "https://s3.central.data.destination-earth.eu"
USERNAME = ''
PASSWORD = ''

workflow = "data-harvest"
identifier = "S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE"
order_body_custom_bucket = {
        "Name": "DEDL - Hook tutorial - temporary storage - " + workflow + " - " + test_run_id,
        "WorkflowName": workflow,
        "IdentifierList": [identifier],
        "WorkflowOptions":[
            {"Name": "output_storage", "Value": "TEMPORARY"},
            {"Name": "input_catalogue_type", "Value": "STAC"},
            {"Name": "input_catalogue_url", "Value": "https://hda.data.destination-earth.eu/stac"},
            {"Name": "input_catalogue_collection", "Value": "EO.ESA.DAT.SENTINEL-2.MSI.L2A"},
            {"Name": "source_client_id", "Value": "hda-public"},
            {"Name": "source_client_secret", "Value": ""},
            {"Name": "source_username", "Value": USERNAME},
            {"Name": "source_password", "Value": PASSWORD},
            {"Name": "source_realm", "Value": "desp"},
            {"Name": "source_server_url", "Value": "https://identity.data.destination-earth.eu/auth"}    
        ]
    }
request = requests.post(service_root_url+"BatchOrder/OData.CSC.Order",
                            json.dumps(order_body_custom_bucket),headers=api_headers)
JSON(request.json(), indent=2)

<IPython.core.display.JSON object>

## Get status of your order
* When 'Status' is 'completed' get your 'Id' (in this case 13832) and follow to the next step.

In [13]:
result = requests.get(service_root_url + "ProductionOrders?$filter=(endswith(Name,'" + test_run_id + "'))", headers=api_headers).json()
print(json.dumps(result, indent=2))

{
  "@odata.context": "$metadata#ProductionOrder/$entity",
  "value": [
    {
      "Id": "13832",
      "Status": "completed",
      "StatusMessage": "requested output product is available",
      "SubmissionDate": "2024-05-14T12:33:18.113Z",
      "Name": "DEDL - Hook tutorial - temporary storage - data-harvest - test_hook",
      "EstimatedDate": "2024-05-14T12:37:17.914Z",
      "InputProductReference": {
        "Reference": "S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE",
        "ContentDate": null
      },
      "WorkflowOptions": [
        {
          "Name": "brand",
          "Value": "dedl"
        },
        {
          "Name": "platform",
          "Value": "creodias"
        },
        {
          "Name": "version",
          "Value": "0.0.1"
        },
        {
          "Name": "output_storage",
          "Value": "TEMPORARY"
        },
        {
          "Name": "input_catalogue_type",
          "Value": "STAC"
        },
        {
          "Nam

## Download data  
* Put your 'Id" next to "BatchOrder" and link to download product will be provided.

In [16]:
response = requests.get('https://odp.data.destination-earth.eu/odata/v1/BatchOrder(13832)/Products', headers=api_headers)
download_json = json.dumps(response.json(), indent=2)
print(download_json)

{
  "@odata.context": "#metadata/OData.CSC.BatchorderItem",
  "value": [
    {
      "Id": 19321,
      "BatchOrderId": 13832,
      "InputProductReference": "S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE",
      "SubmissionDate": "2024-05-14T12:33:18.061Z",
      "Status": "completed",
      "ProcessedName": "S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE",
      "ProcessedSize": 269952793,
      "OutputUUID": null,
      "DownloadLink": "https://s3.central.data.destination-earth.eu/swift/v1/tmp-storage/20240514_19321_VyMp5YNv.zip?temp_url_sig=966aff0a4256f7e7465ecdc98fd692afb65d427b&temp_url_expires=1716899838",
      "NotificationStatus": null,
      "CompletedDate": null
    }
  ]
}
